In [1]:
pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 7.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importing third-party libraries
import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical operations
import unidecode
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='IPython.core.pylabtools')

# Loading the admission results dataset
admission_df = pd.read_csv('/kaggle/input/uni-admission-results-2016-2024/UNI Admission Results 2016 - 2024.csv')

admission_df

,IDHASH,COLEGIO,COLEGIO_DEPA,COLEGIO_PROV,COLEGIO_DIST,COLEGIO_PAIS,COLEGIO_ANIO_EGRESO,ESPECIALIDAD,ANIO_POSTULA,CICLO_POSTULA,...,DOMICILIO_DIST,ANIO_NACIMIENTO,NACIMIENTO_PAIS,NACIMIENTO_DEPA,NACIMIENTO_PROV,NACIMIENTO_DIST,SEXO,CALIF_FINAL,INGRESO,MODALIDAD
0,E4287C2FE19F63C5E6641955147E36684A5A2FF8064676...,LA DIVINA PROVIDENCIA,LIMA,LIMA,SURQUILLO,PERÚ,2020,INGENIERÍA DE SISTEMAS,2021,1,...,SAN JUAN DE MIRAFLORES,2004,PERÚ,LIMA,LIMA,VILLA EL SALVADOR,MASCULINO,3.110,NO,EXTRAORDINARIO1 - DEPORTISTAS CALIFICADOS DE A...
1,2DC37F0B9727B6591EC72D7A942647797A200F45D47C9E...,86019 LA LIBERTAD,ÁNCASH,HUARAZ,HUARAZ,PERÚ,2017,INGENIERÍA DE TELECOMUNICACIONES,2021,1,...,HUARAZ,2001,PERÚ,ÁNCASH,HUARAZ,HUARAZ,FEMENINO,3.580,NO,ORDINARIO
2,3B8677B90781D7BB8F2F967C05FA2DBBE153BBB682DF05...,0113 DANIEL ALOMIAS ROBLES,LIMA,LIMA,SAN JUAN DE LURIGANCHO,PERÚ,2016,INGENIERÍA MECÁNICA,2021,1,...,SAN JUAN DE LURIGANCHO,2000,PERÚ,LIMA,LIMA,JESÚS MARÍA,MASCULINO,8.040,NO,ORDINARIO
3,FA366704D9E9F6FB5E5F55C1FB0CEEE973C626A5616F55...,SEBASTIAN SALAZAR BONDY,LIMA,LIMA,SANTA ANITA,PERÚ,2018,INGENIERÍA ELECTRÓNICA,2021,1,...,SANTA ANITA,2002,PERÚ,LIMA,LIMA,LIMA,MASCULINO,10.820,NO,EXTRAORDINARIO2 – INGRESO DIRECTO CEPRE
4,E31CF8F30F3AE60B3D8A14F6E1020E9AD26EE975F0823B...,TRILCE LOS OLIVOS,LIMA,LIMA,SAN MARTÍN DE PORRES,PERÚ,2020,ARQUITECTURA,2021,1,...,INDEPENDENCIA,2004,PERÚ,LIMA,LIMA,COMAS,FEMENINO,6.210,NO,ORDINARIO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42511,370728AB03E7ECDE69265AC64B0A682887B0BA3E40644F...,BARTOLOME HERRERA,LIMA,LIMA,SAN MIGUEL,NaN,2019,INGENIERÍA CIVIL,2023,1,...,LA PERLA,2002,PERÚ,CALLAO,PROV CONST DEL CALLAO,BELLAVISTA,MASCULINO,5.906,NO,ORDINARIO
42512,C03C7AC74210DA0F4862BFEDAE45CF3BB5AE9813FCF409...,FE Y ALEGRIA 59,CALLAO,PROV CONST DEL CALLAO,VENTANILLA,NaN,2021,INGENIERÍA CIVIL,2023,1,...,VENTANILLA,2004,PERÚ,CALLAO,PROV CONST DEL CALLAO,CALLAO,FEMENINO,2.621,NO,ORDINARIO
42513,E6337F43BAEF0F59EDF11F6F16A47A908717DE288B7886...,CIENCIAS,AYACUCHO,HUAMANGA,SAN JUAN BAUTISTA,NaN,2022,ARQUITECTURA,2023,1,...,AYACUCHO,2006,PERÚ,AYACUCHO,HUAMANGA,AYACUCHO,FEMENINO,7.974,NO,ORDINARIO
42514,13F7CD09C661B02F61218445885DA7C4FDBD87392F7522...,SAN CARLOS,LIMA,LIMA,LURIGANCHO,NaN,2021,ARQUITECTURA,2023,1,...,LURIGANCHO,2004,PERÚ,LIMA,LIMA,MIRAFLORES,FEMENINO,9.477,NO,ORDINARIO


In [3]:
admission_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42516 entries, 0 to 42515
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   IDHASH               42516 non-null  object 
 1   COLEGIO              42516 non-null  object 
 2   COLEGIO_DEPA         42290 non-null  object 
 3   COLEGIO_PROV         42288 non-null  object 
 4   COLEGIO_DIST         42288 non-null  object 
 5   COLEGIO_PAIS         18942 non-null  object 
 6   COLEGIO_ANIO_EGRESO  42516 non-null  int64  
 7   ESPECIALIDAD         42461 non-null  object 
 8   ANIO_POSTULA         42516 non-null  int64  
 9   CICLO_POSTULA        42516 non-null  int64  
 10  DOMICILIO_DEPA       42516 non-null  object 
 11  DOMICILIO_PROV       42516 non-null  object 
 12  DOMICILIO_DIST       42516 non-null  object 
 13  ANIO_NACIMIENTO      42516 non-null  int64  
 14  NACIMIENTO_PAIS      42516 non-null  object 
 15  NACIMIENTO_DEPA      42314 non-null 

In [4]:
# List of modalities to keep
modalities_to_keep = [
    "ORDINARIO",
    "INGRESO ESCOLAR NACIONAL",
    "EXTRAORDINARIO 2 – INGRESO DIRECTO CEPRE",
    "EXTRAORDINARIO INGRESO DIRECTO CEPRE-UNI",
    "EXTRAORDINARIO INGRESO DIRECTO CEPRE-UNI O CEPRE-UNI INTENSIVO",
    "EXTRAORDINARIO2 – INGRESO DIRECTO CEPRE",
    "TALENTO BECA 18",
    "EXTRAORDINARIO - DOS PRIMEROS ALUMNOS",
    "EXTRAORDINARIO 1 - DOS PRIMEROS ALUMNOS",
    "EXTRAORDINARIO1 - DOS PRIMEROS ALUMNOS",
    "EXTRAORDINARIO - VÍCTIMA DEL TERRORISMO (iniciar estudios)",
    "EXTRAORDINARIO 1 - VÍCTIMA DEL TERRORISMO (iniciar estudios)",
    "EXTRAORDINARIO - PERSONA CON DISCAPACIDAD (iniciar estudios)",
    "EXTRAORDINARIO 1 - PERSONAS CON DISCAPACIDAD (iniciar estudios)",
    "EXTRAORDINARIO 1 - DEPORTISTAS CALIFICADOS DE ALTO NIVEL( Iniciar estudios)",
    "EXTRAORDINARIO - DEPORTISTA CALIFICADO DE ALTO NIVEL( Iniciar estudios)",
    "EXTRAORDINARIO1 - PERSONAS CON DISCAPACIDAD (iniciar estudios)",
    "EXTRAORDINARIO1 - DEPORTISTAS CALIFICADOS DE ALTO NIVEL( Iniciar estudios)"
]

# Keep only the records that have those modalities
admission_df = admission_df[admission_df['MODALIDAD'].isin(modalities_to_keep)]

# Verify the changes
admission_df

,IDHASH,COLEGIO,COLEGIO_DEPA,COLEGIO_PROV,COLEGIO_DIST,COLEGIO_PAIS,COLEGIO_ANIO_EGRESO,ESPECIALIDAD,ANIO_POSTULA,CICLO_POSTULA,...,DOMICILIO_DIST,ANIO_NACIMIENTO,NACIMIENTO_PAIS,NACIMIENTO_DEPA,NACIMIENTO_PROV,NACIMIENTO_DIST,SEXO,CALIF_FINAL,INGRESO,MODALIDAD
0,E4287C2FE19F63C5E6641955147E36684A5A2FF8064676...,LA DIVINA PROVIDENCIA,LIMA,LIMA,SURQUILLO,PERÚ,2020,INGENIERÍA DE SISTEMAS,2021,1,...,SAN JUAN DE MIRAFLORES,2004,PERÚ,LIMA,LIMA,VILLA EL SALVADOR,MASCULINO,3.110,NO,EXTRAORDINARIO1 - DEPORTISTAS CALIFICADOS DE A...
1,2DC37F0B9727B6591EC72D7A942647797A200F45D47C9E...,86019 LA LIBERTAD,ÁNCASH,HUARAZ,HUARAZ,PERÚ,2017,INGENIERÍA DE TELECOMUNICACIONES,2021,1,...,HUARAZ,2001,PERÚ,ÁNCASH,HUARAZ,HUARAZ,FEMENINO,3.580,NO,ORDINARIO
2,3B8677B90781D7BB8F2F967C05FA2DBBE153BBB682DF05...,0113 DANIEL ALOMIAS ROBLES,LIMA,LIMA,SAN JUAN DE LURIGANCHO,PERÚ,2016,INGENIERÍA MECÁNICA,2021,1,...,SAN JUAN DE LURIGANCHO,2000,PERÚ,LIMA,LIMA,JESÚS MARÍA,MASCULINO,8.040,NO,ORDINARIO
3,FA366704D9E9F6FB5E5F55C1FB0CEEE973C626A5616F55...,SEBASTIAN SALAZAR BONDY,LIMA,LIMA,SANTA ANITA,PERÚ,2018,INGENIERÍA ELECTRÓNICA,2021,1,...,SANTA ANITA,2002,PERÚ,LIMA,LIMA,LIMA,MASCULINO,10.820,NO,EXTRAORDINARIO2 – INGRESO DIRECTO CEPRE
4,E31CF8F30F3AE60B3D8A14F6E1020E9AD26EE975F0823B...,TRILCE LOS OLIVOS,LIMA,LIMA,SAN MARTÍN DE PORRES,PERÚ,2020,ARQUITECTURA,2021,1,...,INDEPENDENCIA,2004,PERÚ,LIMA,LIMA,COMAS,FEMENINO,6.210,NO,ORDINARIO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42511,370728AB03E7ECDE69265AC64B0A682887B0BA3E40644F...,BARTOLOME HERRERA,LIMA,LIMA,SAN MIGUEL,NaN,2019,INGENIERÍA CIVIL,2023,1,...,LA PERLA,2002,PERÚ,CALLAO,PROV CONST DEL CALLAO,BELLAVISTA,MASCULINO,5.906,NO,ORDINARIO
42512,C03C7AC74210DA0F4862BFEDAE45CF3BB5AE9813FCF409...,FE Y ALEGRIA 59,CALLAO,PROV CONST DEL CALLAO,VENTANILLA,NaN,2021,INGENIERÍA CIVIL,2023,1,...,VENTANILLA,2004,PERÚ,CALLAO,PROV CONST DEL CALLAO,CALLAO,FEMENINO,2.621,NO,ORDINARIO
42513,E6337F43BAEF0F59EDF11F6F16A47A908717DE288B7886...,CIENCIAS,AYACUCHO,HUAMANGA,SAN JUAN BAUTISTA,NaN,2022,ARQUITECTURA,2023,1,...,AYACUCHO,2006,PERÚ,AYACUCHO,HUAMANGA,AYACUCHO,FEMENINO,7.974,NO,ORDINARIO
42514,13F7CD09C661B02F61218445885DA7C4FDBD87392F7522...,SAN CARLOS,LIMA,LIMA,LURIGANCHO,NaN,2021,ARQUITECTURA,2023,1,...,LURIGANCHO,2004,PERÚ,LIMA,LIMA,MIRAFLORES,FEMENINO,9.477,NO,ORDINARIO


In [5]:
admission_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41900 entries, 0 to 42515
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   IDHASH               41900 non-null  object 
 1   COLEGIO              41900 non-null  object 
 2   COLEGIO_DEPA         41900 non-null  object 
 3   COLEGIO_PROV         41900 non-null  object 
 4   COLEGIO_DIST         41900 non-null  object 
 5   COLEGIO_PAIS         18625 non-null  object 
 6   COLEGIO_ANIO_EGRESO  41900 non-null  int64  
 7   ESPECIALIDAD         41900 non-null  object 
 8   ANIO_POSTULA         41900 non-null  int64  
 9   CICLO_POSTULA        41900 non-null  int64  
 10  DOMICILIO_DEPA       41900 non-null  object 
 11  DOMICILIO_PROV       41900 non-null  object 
 12  DOMICILIO_DIST       41900 non-null  object 
 13  ANIO_NACIMIENTO      41900 non-null  int64  
 14  NACIMIENTO_PAIS      41900 non-null  object 
 15  NACIMIENTO_DEPA      41706 non-null  obje

In [6]:
# Function to remove accents from the entire DataFrame
def remove_accents_from_df(df):
    return df.map(lambda text: unidecode.unidecode(text) if isinstance(text, str) else text)

In [7]:
# Apply the function to remove accents from the entire DataFrame
admission_df = remove_accents_from_df(admission_df)

# List of countries to remove
countries_to_remove = ['VENEZUELA', 'ARGENTINA', 'CHILE', 'CANADA', 'COLOMBIA', 'ECUADOR', 'ITALIA']

# Remove records that belong to those countries
admission_df = admission_df[~admission_df['COLEGIO_DEPA'].isin(countries_to_remove)]

In [8]:
admission_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41885 entries, 0 to 42515
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   IDHASH               41885 non-null  object 
 1   COLEGIO              41885 non-null  object 
 2   COLEGIO_DEPA         41885 non-null  object 
 3   COLEGIO_PROV         41885 non-null  object 
 4   COLEGIO_DIST         41885 non-null  object 
 5   COLEGIO_PAIS         18617 non-null  object 
 6   COLEGIO_ANIO_EGRESO  41885 non-null  int64  
 7   ESPECIALIDAD         41885 non-null  object 
 8   ANIO_POSTULA         41885 non-null  int64  
 9   CICLO_POSTULA        41885 non-null  int64  
 10  DOMICILIO_DEPA       41885 non-null  object 
 11  DOMICILIO_PROV       41885 non-null  object 
 12  DOMICILIO_DIST       41885 non-null  object 
 13  ANIO_NACIMIENTO      41885 non-null  int64  
 14  NACIMIENTO_PAIS      41885 non-null  object 
 15  NACIMIENTO_DEPA      41696 non-null  obje

In [9]:
# Fill the empty values in 'COLEGIO_PAIS' with the string 'PERU'
admission_df['COLEGIO_PAIS'] = admission_df['COLEGIO_PAIS'].fillna('PERU')

# Remove records where 'NACIMIENTO_PAIS' is not 'PERU'
admission_df = admission_df[admission_df['NACIMIENTO_PAIS'] == 'PERU']

In [10]:
admission_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41670 entries, 0 to 42515
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   IDHASH               41670 non-null  object 
 1   COLEGIO              41670 non-null  object 
 2   COLEGIO_DEPA         41670 non-null  object 
 3   COLEGIO_PROV         41670 non-null  object 
 4   COLEGIO_DIST         41670 non-null  object 
 5   COLEGIO_PAIS         41670 non-null  object 
 6   COLEGIO_ANIO_EGRESO  41670 non-null  int64  
 7   ESPECIALIDAD         41670 non-null  object 
 8   ANIO_POSTULA         41670 non-null  int64  
 9   CICLO_POSTULA        41670 non-null  int64  
 10  DOMICILIO_DEPA       41670 non-null  object 
 11  DOMICILIO_PROV       41670 non-null  object 
 12  DOMICILIO_DIST       41670 non-null  object 
 13  ANIO_NACIMIENTO      41670 non-null  int64  
 14  NACIMIENTO_PAIS      41670 non-null  object 
 15  NACIMIENTO_DEPA      41670 non-null  obje

In [11]:
file_path = '/kaggle/working/admision_df.xlsx'
admission_df.to_excel(file_path, index=False)